In [1]:
# !pip3 install Pillow
# !pip3 install pytesseract
# !pip3 install webdriver-manager
# !pip3 install ddddocr
# !pip install opencv-python==4.5.5.64

In [2]:
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
import cv2, time, ddddocr
import numpy as np
from datetime import datetime, timedelta

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
# def old_captchaCrack(driver: webdriver, ID:str) -> str:
#     img_path = "prefix.png"
#     capture = "capture.png"
#     driver.save_screenshot(img_path)
#     element = driver.find_element(By.ID, ID)
#     left = element.location['x']
#     right = element.location['x'] + element.size["width"]
#     top = element.location['y']
#     bottom = element.location['y'] + element.size["height"]

#     from PIL import Image
#     img = Image.open(img_path)
#     img = img.crop((left, top, right, bottom))
    
# #     # 灰階處理
# #     gray = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
# #     _, threshold = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)
# #     # 雜訊處理
# #     noise = np.random.normal(0, 15, threshold.shape)
# #     noise = np.clip(threshold +noise, 0, 255).astype('uint8')
# #     guassian = cv2.blur(noise, (3, 3))
# #     img = Image.fromarray(guassian)

#     imgry = img.convert('L')
#     _, threshold = cv2.threshold(np.array(imgry), 140, 255, cv2.THRESH_BINARY)
#     img = Image.fromarray(threshold)
    
#     # 最後存回 PIL Image
#     img.save(capture)
#     ocr = ddddocr.DdddOcr()
#     return ocr.classification(img)

In [4]:
def captchaCrack(driver: webdriver, ID:str) -> str:
    img_path = "cv2.png"
    capture = "capture.png"
    element = driver.find_element(By.ID, ID)
    data = element.screenshot_as_png
    decoded = cv2.imdecode(np.frombuffer(data, np.uint8), 1)
    cv2.imwrite(img_path, decoded)
    from PIL import Image
    img = Image.open(img_path)
    imgry = img.convert('L')
    _, threshold = cv2.threshold(np.array(imgry), 140, 255, cv2.THRESH_BINARY)
    img = Image.fromarray(threshold)
    
#     # 灰階處理
#     gray = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
#     _, threshold = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)
#     # 雜訊處理
#     noise = np.random.normal(0, 15, threshold.shape)
#     noise = np.clip(threshold +noise, 0, 255).astype('uint8')
#     guassian = cv2.blur(noise, (3, 3))
#     img = Image.fromarray(guassian)

    # 最後存回 PIL Image
    img.save(capture)
    ocr = ddddocr.DdddOcr()
    return ocr.classification(img)

def isElementPresent(driver: webdriver, key:str):
    try:
        element = driver.find_element(By.NAME, key)
    except NoSuchElementException as e:
        return False
    return element

# 判斷是否有彈出框
def alert_is_present(driver: webdriver):
    try:
        alert = driver.switch_to.alert
        alert.text
        return alert
    except:
        return False

In [5]:
def login(driver: webdriver):
    KEY_ID = 'F1234852656'
    E_mail = "abc123456@gmail.com"
    # 抓取下拉選單元件
    select = Select(driver.find_element(By.NAME, 'ctl00$ContentPlaceHolder1$apply_nation'))
    select.select_by_index(1)
    time.sleep(0.5)
    # Writting 
    driver.find_element(By.NAME, 'ctl00$ContentPlaceHolder1$apply_sid').send_keys(KEY_ID)
    driver.find_element(By.NAME, 'ctl00$ContentPlaceHolder1$apply_email').send_keys(E_mail)
    driver.find_element(By.NAME, 'ctl00$ContentPlaceHolder1$vcode').send_keys(captchaCrack(driver, "ContentPlaceHolder1_imgcode"))
    driver.find_element(By.NAME, 'ctl00$ContentPlaceHolder1$btnappok').click()
    
def submit(driver: webdriver):
    driver.find_element(By.NAME, 'ctl00$ContentPlaceHolder1$btnstepdown').click()
    driver.find_element(By.NAME, 'ctl00$ContentPlaceHolder1$vcode2').send_keys(captchaCrack(driver, "ContentPlaceHolder1_imgcode2"))
    driver.find_element(By.NAME, 'ctl00$ContentPlaceHolder1$btnstep2downnext').click()
    
def result(driver: webdriver):
    driver.find_element(By.NAME, 'ctl00$ContentPlaceHolder1$vcode').send_keys(captchaCrack(driver, "ContentPlaceHolder1_imgcode"))
    driver.find_element(By.NAME, 'ctl00$ContentPlaceHolder1$btnsave').click()
    time.sleep(2)

def first_step(driver: webdriver):
    login(driver)
    while True:
        # 如果有彈出框 點擊確定
        if alert_is_present(driver):
            driver.switch_to.alert.accept()
        btnupd = isElementPresent(driver, 'ctl00$ContentPlaceHolder1$New_List$ctl00$btnupd')
        if not btnupd:
            driver.find_element(By.XPATH, "//a[@href='apply_2_1.aspx']").click();
            login(driver)
        else:
            break

    btnupd.click()
    time.sleep(1)
    if alert_is_present(driver):
        driver.switch_to.alert.accept()
        
def two_step(driver: webdriver):
    submit(driver)
    while True:
        # 如果有彈出框 點擊確定
        if alert_is_present(driver):
            driver.switch_to.alert.accept()
        btnsave = isElementPresent(driver, "ctl00$ContentPlaceHolder1$btnsave")
        if not btnsave:
            driver.refresh()
            submit(driver)
        else:
            break
            
def final_step(driver: webdriver):
    result(driver)
    while True:
        # 如果有彈出框 點擊確定
        if alert_is_present(driver):
            driver.switch_to.alert.accept()
        btnPrint = isElementPresent(driver, "ctl00$ContentPlaceHolder1$btnPrint")
        if not btnPrint:
            driver.find_element(By.NAME, 'ctl00$ContentPlaceHolder1$vcode').clear()
            result(driver)
        else:
            break

In [6]:
while True:
    today_now = datetime.now()
    today_at = today_now.replace(hour=8, minute=15, second=0)
    if today_now == today_at:
        start_time = time.time()
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get("https://npm.cpami.gov.tw/apply_2_1.aspx")
        first_step(driver)
        two_step(driver)
        final_step(driver)
        print("Cost time:",time.time() - start_time)
        break

欢迎使用ddddocr，本项目专注带动行业内卷，个人博客:wenanzhe.com
训练数据支持来源于:http://146.56.204.113:19199/preview
爬虫框架feapder可快速一键接入，快速开启爬虫之旅：https://github.com/Boris-code/feapder
谷歌reCaptcha验证码 / hCaptcha验证码 / funCaptcha验证码商业级识别接口：https://yescaptcha.com/i/NSwk7i
欢迎使用ddddocr，本项目专注带动行业内卷，个人博客:wenanzhe.com
训练数据支持来源于:http://146.56.204.113:19199/preview
爬虫框架feapder可快速一键接入，快速开启爬虫之旅：https://github.com/Boris-code/feapder
谷歌reCaptcha验证码 / hCaptcha验证码 / funCaptcha验证码商业级识别接口：https://yescaptcha.com/i/NSwk7i
欢迎使用ddddocr，本项目专注带动行业内卷，个人博客:wenanzhe.com
训练数据支持来源于:http://146.56.204.113:19199/preview
爬虫框架feapder可快速一键接入，快速开启爬虫之旅：https://github.com/Boris-code/feapder
谷歌reCaptcha验证码 / hCaptcha验证码 / funCaptcha验证码商业级识别接口：https://yescaptcha.com/i/NSwk7i
欢迎使用ddddocr，本项目专注带动行业内卷，个人博客:wenanzhe.com
训练数据支持来源于:http://146.56.204.113:19199/preview
爬虫框架feapder可快速一键接入，快速开启爬虫之旅：https://github.com/Boris-code/feapder
谷歌reCaptcha验证码 / hCaptcha验证码 / funCaptcha验证码商业级识别接口：https://yescaptcha.com/i/NSwk7i
Cost time: 12.798277139663696
